In [1]:
!pip install llama-index

  Obtaining dependency information for llama-index from https://files.pythonhosted.org/packages/2b/2b/0d4914384513dd15ea60a52a0f6b380b7dee2be7e2049e4c0986ed863305/llama_index-0.8.20-py3-none-any.whl.metadata
  Obtaining dependency information for sqlalchemy>=2.0.15 from https://files.pythonhosted.org/packages/91/2b/92aadcea86b9ebd681de0b6b2cbfa75193227e607893cfb5feea0cefc461/SQLAlchemy-2.0.20-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for fsspec>=2023.5.0 from https://files.pythonhosted.org/packages/3a/9f/b40e8e5be886143379000af5fc0c675352d59e82fd869d24bf784161dc77/fsspec-2023.9.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.3/752.3 kB 1.5 MB/s eta 0:00:001.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 2.7 MB/s eta 0:00:00MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 2.1 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: sqlalchemy
 

## Setting API KEY

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.environ.get('OPENAI_API_KEY')

## build a simple vector store index

In [4]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('countries').load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [6]:
print(query_engine.query("what is the culture of India"))

India's culture is known for its incredible diversity, with a variety of languages, religions, and traditions. It is often referred to as the "Land of Diversity" due to the numerous ethnic groups that call it home. The country's cultural landscape is heavily influenced by its religious diversity, with major religions such as Hinduism, Buddhism, Jainism, and Sikhism originating in India. Art and architecture are integral to Indian culture, with intricate temple sculptures, colorful frescoes, and stunning architectural wonders like the Taj Mahal. Indian cuisine is famous worldwide for its diverse flavors and regional specialties, with spices playing a central role in cooking. Traditional Indian clothing varies across regions, with sarees, dhotis, and kurta-pajamas being commonly worn. India is also renowned for its vibrant festivals, such as Diwali (Festival of Lights) and Holi (Festival of Colors), which are celebrated with enthusiasm and fervor. India's history dates back to the ancien

In [7]:
print(query_engine.query("List down Festivals of USA with its's date if possible"))

Independence Day - July 4th
Thanksgiving - Fourth Thursday in November
Halloween - October 31st
Christmas - December 25th
Labor Day - First Monday in September


In [9]:
print(query_engine.query("what is the language of Pakistan"))

The language of Pakistan is Urdu.


## Storing

In [10]:
index.storage_context.persist('countries-index')

## Loading Index

In [11]:
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir='countries-index')
# load index
index = load_index_from_storage(storage_context)

In [12]:
query_engine_new = index.as_query_engine()

In [13]:
print(query_engine_new.query("List down Festivals of USA with its's date if possible"))

Independence Day - July 4th
Thanksgiving - Fourth Thursday in November
Halloween - October 31st
Christmas - December 25th
Labor Day - First Monday in September


# Customization

## parse documents into smaller chunks

In [14]:
from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(chunk_size=1000)
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('countries').load_data()
index = VectorStoreIndex.from_documents(documents, service_context=service_context)
query_engine = index.as_query_engine()
response = query_engine.query("List the Cuisines of England?")
print(response)

English cuisine includes classics like fish and chips, roast dinners, and afternoon tea. The country also has a diverse culinary scene influenced by immigration, with a variety of international restaurants.


## use a different vector store

In [15]:
import chromadb
from llama_index.vector_stores import ChromaVectorStore
from llama_index import StorageContext


chroma_client = chromadb.PersistentClient()
chroma_collection = chroma_client.create_collection('countries_collection')
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [17]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('countries').load_data()
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)
query_engine = index.as_query_engine()
response = query_engine.query("List out the Cuisines of Pakistan in bullet points?")
print(response)

- Biryani
- Kebabs
- Nihari
- Naan and roti (various types of bread)


## retrieve more context when I query

In [18]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('countries').load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine(similarity_top=5)
response = query_engine.query("Describe about the Festivals of India?")
print(response)

India is renowned for its vibrant festivals, each celebrated with enthusiasm and fervor. Some of the most significant festivals in India include Diwali, Holi, Eid, Durga Puja, and Christmas.

Diwali, also known as the Festival of Lights, celebrates the victory of light over darkness and good over evil. It involves lighting lamps, bursting fireworks, exchanging gifts, and enjoying sweets.

Holi, known as the Festival of Colors, is a joyful celebration marked by people throwing colored powders and water at each other. It signifies the arrival of spring.

Eid is celebrated by Muslims in India with prayers, feasts, and the exchange of gifts. These festivals hold special significance in Islamic culture.

Durga Puja is a major Hindu festival celebrated predominantly in West Bengal. It honors the goddess Durga and features elaborate decorations, cultural performances, and processions.

Christmas is celebrated by Christians and non-Christians alike in India. It involves decorated trees, midnig

## use a different LLM

In [25]:
from llama_index import ServiceContext,LLMPredictor
from langchain.chat_models import ChatOpenAI

llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

In [26]:
documents = SimpleDirectoryReader('countries').load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine(similarity_top=5)
response = query_engine.query("Describe about the Festivals of USA?")
print(response)

The festivals of the United States (USA) are diverse and celebrated with enthusiasm. One significant festival is Independence Day, which commemorates the Declaration of Independence and is celebrated on the Fourth of July. It involves fireworks, parades, barbecues, and patriotic displays. Thanksgiving is another important festival, observed on the fourth Thursday in November. It is a time for families to gather and give thanks, with a traditional feast that includes roast turkey and pumpkin pie. Halloween is a fun holiday celebrated on October 31st, with costume parties, trick-or-treating, and haunted house visits. Christmas is widely celebrated with decorations, gift-giving, and gatherings, and is marked by festive lights and decorations in many cities. Labor Day, observed on the first Monday in September, honors the contributions of workers and is celebrated with picnics and parades. These festivals reflect the cultural and historical significance of the United States.


## Using Chatbot Instead of Q&A

In [28]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('countries').load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_chat_engine()
response = query_engine.query("What are the festivals of England?")
print(response)

England celebrates a variety of festivals throughout the year. Some of the popular festivals in England include:

1. Notting Hill Carnival: Held in August, it is the largest street festival in Europe, celebrating Caribbean culture with colorful costumes, music, and dance.

2. Glastonbury Festival: A famous music festival held in June, featuring a wide range of music genres and attracting thousands of music lovers from around the world.

3. Bonfire Night: Also known as Guy Fawkes Night, it is celebrated on November 5th to commemorate the failed Gunpowder Plot of 1605. Fireworks displays and bonfires are common during this festival.

4. Christmas: Celebrated on December 25th, Christmas is a major festival in England. It is a time for family gatherings, exchanging gifts, and enjoying festive meals.

5. Easter: A Christian festival celebrated in March or April, Easter marks the resurrection of Jesus Christ. It is a time for church services, Easter egg hunts, and family meals.

6. St. Patri

In [29]:
response = query_engine.chat("Oh interesting, what are their Cuisines")
print(response)

England has a diverse culinary scene with a variety of traditional and modern cuisines. Here are some popular English cuisines:

1. Fish and Chips: This is a classic English dish consisting of deep-fried battered fish (usually cod or haddock) served with chips (thick-cut fries). It is often accompanied by mushy peas and tartar sauce.

2. Roast Beef and Yorkshire Pudding: A traditional Sunday roast, this dish features roasted beef served with Yorkshire pudding (a savory baked batter), roasted potatoes, vegetables, and gravy.

3. Full English Breakfast: A hearty breakfast meal that typically includes bacon, eggs, sausages, baked beans, grilled tomatoes, mushrooms, and toast. It is often served with black pudding and a cup of tea.

4. Afternoon Tea: A quintessential English tradition, afternoon tea is a light meal typically served between lunch and dinner. It includes a selection of finger sandwiches, scones with clotted cream and jam, cakes, and a pot of tea.

5. Cornish Pasty: Originati

## Customizing Prompt

In [31]:
from llama_index import Prompt

# Define a custom prompt
template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question and each answer should start with code word Mohammed Shaneeb AI: {query_str}\n"
)
qa_template = Prompt(template)

# Use the custom prompt when querying
query_engine = index.as_query_engine(text_qa_template=qa_template)
response = query_engine.query("what is the culture of Canada?")
print(response)


Mohammed Shaneeb AI: The culture of Canada is characterized by its cultural diversity, with people from diverse backgrounds and ethnicities coexisting harmoniously. This multiculturalism is a defining feature of Canadian society. Canada is also known for its bilingualism, with English and French as the two official languages. The country has a rich artistic heritage, with contributions to literature, music, film, and visual arts. Indigenous culture is an integral part of Canadian culture, with indigenous art, storytelling, and traditions being celebrated. Canadian cuisine is influenced by both indigenous and immigrant cultures, with iconic dishes like poutine, butter tarts, tourtière, and Nanaimo bars.


## Customizing Embedding 

In [33]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

# Load in a specific embedding model
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2'))

# Create a service context with the custom embedding model
service_context = ServiceContext.from_defaults(embed_model=embed_model)

# Create an index using the service context
new_index = VectorStoreIndex.from_documents(
    documents,
    service_context=service_context,
)

query_engine = index.as_query_engine()
response = query_engine.query("list top 3 cusines of Pakistan")
print(response)


The top three cuisines of Pakistan are biryani, kebabs, and nihari. These dishes are known for their flavorful taste and are popular among both locals and visitors.
